# Practical Exercise: Data Preparation and Visualisation

In [ ]:
import pandas as pd
import numpy as np
import h5py
import os, glob
import re
import itertools
from datetime import datetime, date, time
from six import iteritems



In [ ]:
class Blond(object):
    """
        TODO: figure out how to close files
    """
    
    def __init__(self, date):
        self.date = date
        
    # start(end)_hours_minutes
    def read_clear(self, start_hm, end_hm):
        path_to_files = './data/clear/'
        files_all = next(os.walk(path_to_files))[2]  
        
        for file_name in files_all:            
            pattern = r'(\d{4}-\d{2}-\d{2}T\d{2}-\d{2}-\d{2})'
            regex_obj = re.search(pattern, file_name)
            
            if regex_obj is not None:
                ts_format = regex_obj.group(1)
                file_hm = datetime.strptime(ts_format, '%Y-%m-%dT%H-%M-%S').time()

                if start_hm <= file_hm <= end_hm:
                    yield h5py.File(path_to_files + file_name,'r+')
                         
            
    def read_medal(self, start_hm, end_hm):
        
        """
            each MEDAL is located in a separate folder, thus, we search each folder
        """
        path_to_files = './data/medal*/'
        for folder in glob.glob(path_to_files):            
            files_all = next(os.walk(folder))[2]  
        
            for file_name in files_all:            
                pattern = r'(\d{4}-\d{2}-\d{2}T\d{2}-\d{2}-\d{2})'
                regex_obj = re.search(pattern, file_name)
                
                if regex_obj is not None:
                    ts_format = regex_obj.group(1)
                    file_hm = datetime.strptime(ts_format, '%Y-%m-%dT%H-%M-%S').time()

                    if start_hm <= file_hm <= end_hm:
                        yield h5py.File(folder + file_name, 'r+') 
                        
                    
                     
    def it_center_signal(self, it_medal, signal):
        """no DC_offset in CLEAR data"""   
        for data in it_medal:
            DC_offset = data[signal].attrs['removed_offset']
            data[signal][:] = data[signal][:] + DC_offset
            yield data
            
    def it_calibrate(self, it_data, signal):
        for data in it_data:
            factor = data[signal].attrs['calibration_factor']
            data[signal][:] = (data[signal][:] * factor)
            yield data
        
            
        
    def merge_into_dict(self, it_data, signal):
        
        signal_dict = dict()
        for i, data in enumerate(it_data):   
            """ Dictionary output:
                {0: <HDF5 dataset "current1": shape (15000000,), type "<i2">,
                1: <HDF5 dataset "current1": shape (15000000,), type "<i2">,
                ...
            """
            signal_dict[i] = data[signal] 
        
        return signal_dict     
    
          
     

## 1. Data Reading and Exploration

### Read files and prepare data for plotting

In [ ]:
blond = Blond(date(2018,10,5))

""" Define a timeframe"""
start_hm = time(0,30) # start_hours_minutes
end_hm   = time(1,30)

"""Read MEDAL and CLEAR data """
it_medal = blond.read_medal(start_hm, end_hm)
it_clear = blond.read_clear(start_hm, end_hm)


"""center and calibrate MEDAL data"""
it_data_calibrated = blond.it_calibrate(blond.it_center_signal(it_medal, 'current1'), 'current1')

"""define a signal and prepare data to the format suitable for plotting"""
signal = 'current1'
d = blond.merge_into_dict(it_data_calibrated, 'current1')
data={}
it_merged_data = itertools.chain(*pd.DataFrame(data = d).values)


"""read first 10,000,000 values"""
data['medals_'+signal] = list(itertools.islice(it_merged_data, 10000000) )

In [ ]:
data